## Demo of extracellular simulation of the Hay model with LFPy

Requirements:

- neuron (you should have it installed)
- LFP (`pip install LFPy`)
- neuroplotlib (`pip install git+https://github.com/alejoe91/neuroplotlib.git`)

In [ ]:
import LFPy
import neuron
import os
import MEAutility as mu
import matplotlib.pylab as plt
import numpy as np
import neuroplotlib as nplt
%matplotlib notebook

In [ ]:
import neuron

This function loads the Hay model in LFPy. You can see it takes several parameters from NEURON and you can load morphologies alone or full models.

In [ ]:
def load_hay_model(model_folder, dt, tstart, tstop):
    cwd = os.getcwd()
    os.chdir(model_folder)
    cell = None
    try:
        ##define cell parameters used as input to cell-class
        cellParameters = {
            'morphology': 'morphologies/cell1.asc',
            'templatefile': ['models/L5PCbiophys3.hoc',
                             'models/L5PCtemplate.hoc'],
            'templatename': 'L5PCtemplate',
            'templateargs': 'morphologies/cell1.asc',
            'passive': False,
            'nsegs_method': None,
            'dt': dt,
            'tstart': tstart,
            'tstop': tstop,
            'v_init': -80,
            'celsius': 34,
            'pt3d': True,
        }

        # Initialize cell instance, using the LFPy.Cell class
        neuron.load_mechanisms('mod')
        cell = LFPy.TemplateCell(**cellParameters)
    except:
        print('Failed to load Hay model. Wrong folder?')

    os.chdir(cwd)
    return cell

First, we load the cell and add a somatic current clamp with a current step:

In [ ]:
hay_cell = load_hay_model('L5bPCmodelsEH/', dt=2**-5, tstart=0, tstop=200)
somatic_clamp = LFPy.StimIntElectrode(hay_cell, idx=0, pptype='IClamp', record_current=True, record_potential=True,
                                      amp=0.5, delay=10, dur=150)
somatic_clamp1 = LFPy.StimIntElectrode(hay_cell, idx=0, pptype='IClamp', record_current=True, record_potential=True,
                                       amp=-0.001, delay=10, dur=150)

In [ ]:
hay_cell.simulate(rec_vmem=True, rec_imem=True, variable_dt=False)

In [ ]:
print(len(hay_cell.tvec))
print(len(hay_cell.vmem[0]))
print(len(hay_cell.imem[0]))

In [ ]:
len(somatic_clamp.v)

We can probe the membrane potentials at the same locations as the Hay paper:

In [ ]:
patch_y_locations = [0, 400, 620]

In [ ]:
print(len(hay_cell.tvec))
print(len(hay_cell.vmem[100]))

In [ ]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
for ploc in patch_y_locations:
    idx = hay_cell.get_closest_idx(y=ploc)
    print('Y location:', hay_cell.ymid[idx])
    ax2.plot(hay_cell.tvec, hay_cell.vmem[idx])
ax2b = ax2.twinx()
ax2b.plot(hay_cell.tvec, somatic_clamp.i, color='C0', ls='--', alpha=0.2)

## Extracellular probes

We can use the [MEAutility](https://meautility.readthedocs.io/en/latest/) package to quickly set up extracellular probes (or define new ones):

In [ ]:
neuropixels = mu.return_mea('Neuropixels-128')

In [ ]:
mu.plot_probe(neuropixels)

In [ ]:
sq_mea = mu.return_mea('SqMEA-10-15')

In [ ]:
mu.plot_probe(sq_mea)

Let's use this square MEA (10x10, 15 um pitch). By default, the probe is on the yz plane, so we need to rotate it and move it to 'face' the neuron.

In [ ]:
sq_mea.rotate([0, 1, 0], 90)

In [ ]:
sq_mea.move([0, 0, -50])

In [ ]:
fig3 = plt.figure()
ax3 = fig3.add_subplot()
mu.plot_probe(sq_mea, ax=ax3)
nplt.plot_neuron(hay_cell, plane='xy', ax=ax3)
ax3.axis('equal')

Since we already simulated the cell, we need to re-instantiate it:

In [ ]:
hay_cell = load_hay_model('L5bPCmodelsEH/', dt=2**-5, tstart=0, tstop=50)
somatic_clamp = LFPy.StimIntElectrode(hay_cell, idx=0, pptype='IClamp', record_current=True, 
                                      amp=0.5, delay=10, dur=20)

Now we create an `LFPy.RecExtElectrode` using the `sq_mea` object:

In [ ]:
probe = LFPy.RecExtElectrode(probe=sq_mea)

And we can simulate the cell with the electrodes:

In [ ]:
hay_cell.simulate(variable_dt=True, electrode=probe)

In [ ]:
lfp = probe.LFP * 1000 # in uV

In [ ]:
lfp.shape

In [ ]:
print(len(hay_cell.somav))
print(len(hay_cell.tvec))
print(lfp.shape)

In [ ]:
plt.figure()
plt.plot(hay_cell.tvec, 'o')

In [ ]:
plt.figure()
plt.plot(hay_cell.tvec, lfp[0])

In [ ]:
mu.plot_mea_recording(lfp, sq_mea, colors='b')

In [ ]:
lfp[0].shape

In [ ]:
hay_cell.somav.shape

In [ ]:
neuron.h._ref_t

In [ ]:
plt.figure()
plt.plot(hay_cell.neuron_tvec.as_numpy())